In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install squarify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 5.2 MB/s 
     |████████████████████████████████| 232 kB 52.3 MB/s 
     |████████████████████████████████| 164 kB 63.9 MB/s 
     |████████████████████████████████| 111 kB 68.1 MB/s 
     |████████████████████████████████| 181 kB 60.8 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
     |████████████████████████████████| 4.3 MB 49.9 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 133 kB 72.6 MB/s 
     |████████████████████████████████| 793 kB 68.3 MB/s 
     |████████████████████████████████| 131 kB 73.5 MB/s 
     |████████████████████████████████| 428 kB 64.3 MB/s 
     |████████████████████████████████| 381 kB 58.8 MB/s 
     |████████████████████████████████| 51 kB 8.2 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=76a6c54a780

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=5af49e2b66d94c7ff16af6dcf9296035edaec0937d67044142544d72caffd465
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok
!ngrok authtoken 2A9XTz0MDIsLejrb1W6sjPmrEjg_GfnZojiSm6Bq8tFeDfEj


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%%writefile sales_optimizer_app.py
import streamlit as st 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import squarify

from matplotlib import cm

seed = 100

@st.cache(allow_output_mutation=True)
def load_csv():
  return pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Pakistan Largest Ecommerce Dataset.csv");

df = load_csv()



# st.write('visualize null values')

@st.cache(hash_funcs={matplotlib.figure.Figure: lambda _: None})
def print_NullValues():
  tab20c = cm.get_cmap('tab20c', 2)
  fig = plt.figure(figsize=(10,5))
  sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap=tab20c)
  plt.title("Missing values (in grey) per column", fontweight='bold')
  plt.ylabel("Rows");
  return fig

@st.cache(hash_funcs={matplotlib.figure.Figure: lambda _: None})
def print_transactions_per_order():
  gd1=df.groupby(["increment_id"]).agg({"item_id":"count"})
  gd1["item_id"].value_counts(normalize=True).sort_index()

  fp1=gd1["item_id"].value_counts(normalize=True).sort_index()
  fig = plt.figure(figsize=(10,5))
  plt.bar(fp1.index,fp1.values)
  plt.title("Transactions per order", fontweight="bold")
  plt.xlabel("Transactions")
  plt.ylabel("Percentage of orders")
  plt.axes([0.45,0.4,0.4,0.3],facecolor="white")
  plt.bar(fp1.index,fp1.values)
  plt.xlim([11,75])
  plt.ylim([0,0.001])
  plt.yticks([0.000,0.0005,0.001],[0.000,0.0005,0.001])
  plt.xticks([11,20,40,60,72],[11,20,40,60,72])
  plt.title("Zoom for transactions >10", fontweight="bold");
  return fig

def print_pie_transactions():
    gd1=df.groupby(["increment_id"]).agg({"item_id":"count"})
    order_gt1 = (gd1.item_id > 1).sum()
    order_eq1 = (gd1.item_id == 1).sum()
    fig = plt.figure(figsize=(5,5))
    labels = ['Orders with 1 transaction', 'More than 1 transaction']
    y = [order_eq1, order_gt1]

    # plt.pie(y, labels = labels, autopct='%.2f')
    plt.pie(y, labels = labels, autopct=lambda p:f'{p:.1f}%  ')
    plt.title('Relationship between transactions and order')
    return fig


def print_bar_transactions_per_status():
    fig, ax1 = plt.subplots(figsize=(10,5))
    graph=sns.countplot(ax=ax1, x=df["status"], order=df['status'].value_counts(ascending=False).index, color='#629FCA')
    plt.xticks(fontsize=8,rotation=45)
    plt.ylabel('Count of transactions')
    plt.xlabel('Status')
    plt.title("Count transcations per status", fontweight='bold', fontsize=13);
    for p in graph.patches:
        height = p.get_height()
        plt.text(p.get_x()+p.get_width()/2., height + 4000,height,ha="center",fontsize=8)
    return fig

def print_transactions_timeline():
  # Transactions per month
  df["my"]=pd.to_datetime(df["M-Y"])
  # df["my"]=df["my"].dt.date.apply(lambda x: x.strftime('%Y-%m'))
  my2=df["my"].value_counts().sort_index()
  ticks=my2.index[range(0,26,3)].astype("str")

  dg2=df.groupby(["increment_id"]).agg({"my":"max"})
  my3=dg2["my"].value_counts().sort_index()

  fig = plt.figure(figsize=(10,5))
  #plt.bar(range(0,26),my2.values,width=0.5,alpha=0.3,label="Transactions")
  plt.bar(range(0,26),my2.values,width=0.5,alpha=0.7, label="Transactions")
  plt.xticks(range(0,26,3),ticks)
  plt.title("Transactions and orders per month",  fontweight="bold", fontsize=13)
  plt.xlabel("Year-Month")
  plt.ylabel("Number of transactions")
  plt.plot(range(0,26),my3.values,label="Orders", color='darkblue')
  plt.legend();
  return fig


def distribution_finacial_variables():
  Gt=df.groupby(["increment_id"]).agg({"grand_total":"mean"})

  plt.figure(figsize=(15,12))

  plt.subplot(2,2,1)
  plt.hist(df["price"],bins=1000);
  plt.title("Distribution: price", fontweight='bold')
  plt.xlim([0,50000])
  plt.xlabel('Price')
  plt.ylabel('Transactions')
  plt.axes([0.3,0.65,0.15,0.15],facecolor="white")
  plt.boxplot(df["price"],showfliers=False, vert=False)
  plt.yticks([])

  plt.subplot(2,2,2)
  plt.hist(Gt["grand_total"],bins=10000);
  plt.title("Distribution: grand_total",fontweight='bold')
  plt.xlim([0,100000])
  plt.xlabel('Grand total')
  plt.ylabel('Transactions')
  plt.axes([0.72,0.65,0.15,0.15],facecolor="white")
  plt.boxplot(Gt["grand_total"],showfliers=False, vert=False)
  plt.yticks([])


  plt.subplot(2,2,3)
  plt.hist(df["discount_rate"],bins=200);
  plt.title("Distribution: discount_rate",fontweight='bold')
  plt.xlim([0,100])
  plt.xlabel('Discount rate in %')
  plt.ylabel('Transactions')
  plt.axes([0.3,0.2,0.15,0.15],facecolor="white")
  plt.boxplot(df["discount_rate"],showfliers=False, vert=False)
  plt.yticks([])


  plt.subplot(2,2,4)
  plt.hist(df["qty_ordered"],bins=1000);
  plt.title("Distribution: qty_ordered",fontweight='bold')
  plt.xlim([0,10])
  plt.xlabel('Quantity ordered')
  plt.ylabel('Transactions')
  plt.axes([0.72,0.2,0.15,0.15],facecolor="white")
  plt.boxplot(df["qty_ordered"],showfliers=False, vert=False)
  plt.yticks([]);


def consumption_categories_and_revenue():


    df['grand_item']=df["price"]*df["qty_ordered"] - df["discount_amount"]
    df.category_name_1.fillna('\\N',inplace=True)
    df.category_name_1.replace('\\N',"Others",inplace=True)

    test= pd.DataFrame(df["category_name_1"].value_counts())
    test=test.reset_index()
    # test

    # test.head()

    rev=df.groupby(["category_name_1"]).agg({"grand_item":"sum"}).sort_values(by=["grand_item"],ascending=True)
    rev2=df.groupby(["category_name_1"]).agg({"grand_item":"sum","item_id":"count"}).sort_values(by=["grand_item"],ascending=True)
    rel=rev2["grand_item"]/rev2["item_id"]*100000

    cmap = matplotlib.cm.Blues
    mini=min(test['category_name_1'])
    maxi=max(test['category_name_1'])
    norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)
    colors = [cmap(norm(value)) for value in test['category_name_1']]


    fig = plt.figure(figsize=(15,15))
    plt.subplot(2,1,1)
    squarify.plot(sizes=test["category_name_1"], label=test["index"], alpha=0.8 , color=colors)
    plt.axis('off')
    plt.title("Transactions per category", fontweight='bold')

    plt.subplot(2,1,2)
    plt.barh(range(len(rev)),rev.values.reshape((15,)), label="Revenue")
    plt.yticks(range(len(rev)),rev.index,fontsize=8)
    plt.title("Revenue per category", fontweight='bold')
    plt.xlabel('Revenue (scaled per customer)')
    plt.scatter(rel.values,range(len(rev)),color="red",label="Revenue per Transaction")
    plt.legend();
    return fig


st.sidebar.title("PY-Sales-Optimizer")
pages = ["1. Data Description",
         "2. Data Exploration & Pre-processing",
         "3. RFM Segmentation",
         "4. Marketing Strategy"]
page = st.sidebar.radio("Table of contents", pages)

if page==pages[0]:
    st.title("1. Data Description")

    st.subheader("1.1 Exploration of dataset")

    st.markdown("The underlying dataset consist of online transactions from several Pakistani e-commerce merchants. The transactions were conducted between July, 2016 and August, 2018.")
    
    col1, col2 = st.columns(2)
    col1.metric("Rows (Transactions)", "1.048.575")
    col2.metric("Columns", "26")

    st.markdown("The table below summarizes the corresponding attributes and provides a short description:")

    st.image("/content/drive/MyDrive/Colab Notebooks/figures/table1.jpg", caption="Description of Columns" )

    data_description = st.radio(
     "Explore the dataset",
     ('Dataset preview', 'Explore outliers / extreme values', 'Overview Null Values'))

    if data_description == 'Dataset preview':
      st.dataframe(df.head(100))
    elif data_description == 'Explore outliers / extreme values':
      st.markdown("Additional attention must be paid to extreme values and outliers which can be observed for the numeric variables:")
      st.dataframe(df[["price","qty_ordered","grand_total","discount_amount"]].describe())
    else:
      st.pyplot(print_NullValues())


    # st.markdown("The amounts of missing values per column are negligible (<0.03%). The following graphic provides a visualization of the situation:")
    # st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig1.1.jpg")
    #Extrem values
    # st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig1.2.jpg")
    

elif page==pages[1]:
    
    st.title("2. Data Exploration & Pre-processing")

    data_exploration = st.radio(
     "Data Explorations",
     ('2.1 Transactions, orders & status', '2.2 Price, discount amount & grand total', '2.3 Consumption categories'))

    if data_exploration == '2.1 Transactions, orders & status':
      st.markdown("### 2.1 Transactions, orders & status")
      st.pyplot(print_pie_transactions())
      st.markdown("The 584.314 remaining transactions are resulting from 326.841 single orders. This result is based on the circumstance that 80% of orders only contain one transaction. However, this should not hide the fact that orders with up to 72 transactions exist:")
      st.pyplot(print_transactions_per_order())    
      st.pyplot(print_bar_transactions_per_status())
      st.pyplot(print_transactions_timeline())
  
    elif data_exploration == '2.2 Price, discount amount & grand total':
      st.markdown("### 2.2 Price, discount amount & grand total")

      st.markdown("#### Calculation of grand_total")
      
      st.latex(r'''grand\_total_{order\ i} = \sum_{transactions\ k,i} price_k * qty\_orderd_k - \text{discount amount} 
                ''')
      st.markdown("Observation: Formula only vaild in 80% of the cases")
      
      st.markdown("Dataset has some weaknesses regarding data quality")
      st.markdown(" - excluce transaction, were forumla is not valid")
      st.markdown(" - excluce transaction, with price = 0")
      st.markdown(" - excluce transaction, with discount rate < 0")
      st.markdown(" - excluce transaction, with grand_total < 0")
  

      st.markdown("#### Calculation of discount rate")
      st.latex(r'''discount\_rate  = \frac{discount\_amount}{price * qty\_ordered}''')

      st.markdown("The calculated indicator is significantly correlated with the share of successful product sales as it increases the likelihood for completion and reduces the likelihood for cancellation of a transaction.")
      st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig2.2.1.jpg")

    elif data_exploration == '2.3 Consumption categories':
      st.markdown("### 2.3 Consumption categories")
      st.markdown("there are 14 specified and one Other category to which the ordered products are assigned to.")
      df=df[df["price"]>0]
      df=df[df["grand_total"]>0]
      df=df[df["discount_amount"]>=0]

      st.pyplot(consumption_categories_and_revenue())
      # st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig2.3.1.jpg")

      
    # st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig2.1.1.jpg")


    
elif page==pages[2]:
    st.title("3. RFM segmentation")
    st.markdown("### 3.1 Introduction to RFM segmentation")
    st.markdown("The table below visualizes a potential result of a RFM segmentation and builds the background for the development of a baseline model in the next section:")
    st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig3.1.1.jpg")
    
    st.markdown("### 3.2 RFM scores & baseline model")
    st.markdown("Please be aware that customers with highest “Recency” must receive the lowest Recency-scores. These tables include 60.911 customers and build the basis for the analyses conducted in chapter 3.3.")
    st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig3.2.1a.jpg")
    st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig3.2.1b.jpg")
    
elif page==pages[3]:
    
    st.title("4. Marketing Strategy")
    st.markdown("We aim to uplift company’s sales by approaching customers with a tailor-made product offer. To be successful, it is crucial to approach the ***right customer*** with the ***right offer*** at the ***right point in time***.")
    st.markdown("### 4.1 The right customer")
    st.markdown("Let's look at customer segments that has the highest chance to respond positively to a campaign. The table below provides an overview fo the selected segments and the relevant KPIs:")
    st.image("/content/drive/MyDrive/Colab Notebooks/figures/table4.1.jpg")
    
    st.markdown("### 4.2 The right offer")
    st.markdown("Most of the customers have a tendency to buy products from only 1 or 2 categories, except 'High-Potentials' customers. Hence we tailer the offer to the dominating product category in each group as below.")
    st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig4.2.1.jpg")

    st.markdown("Second part of the strategy relates to the discount amount we offer to our customers. As the effectiveness of discounts may vary considerably depending on the related product category.")
    st.markdown("In general, we expect higher the % of completed orders with a higher % discount offered. For a marketing campaign, we select 5% to 20% as the optimal discount rate as part of the strategy.")
    st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig4.2.2.jpg")

    st.markdown("### 4.3 The right time")
    st.markdown("It's important to launch the marketing campaign when customers are more likely to make a purchase, such as during holidays, seasonal-closings or 'Black-Friday-weeks'. We analysed partterns of purchase behaviour of the selected customer segemnts.")
    st.image("/content/drive/MyDrive/Colab Notebooks/figures/fig4.3.jpg")
    st.markdown("Looking at the amount of transactions, we can see a repeating peak in November of each year. This is related to the “Black Friday”-week in Pakistan and offers a great opportunity for placing our campaign. Another strong increase can be identified end of March where Pakistan celebrates the national day.")
    



Overwriting sales_optimizer_app.py


In [ ]:
!streamlit run --server.port 80 sales_optimizer_app.py&>/dev/null&

In [ ]:
!pgrep streamlit

460


In [ ]:
pub_url = ngrok.connect(port='8501')
pub_url

<NgrokTunnel: "http://ac66-35-243-189-45.ngrok.io" -> "http://localhost:80">